In [170]:
import pandas as pd
import numpy as np


In [171]:
def splitTrainTest(data,percent):
    total=len(data)
    trainTotal=int(total*percent*0.01)
    testTotal=total-trainTotal
    return (data[0:trainTotal],data[trainTotal:total])

In [172]:
class Layer:
    def __init__(self,nNodesCurrent, nNodesNext, activationF):
        self.nodesNo=nNodesCurrent
        self.activations = np.zeros([nNodesCurrent,1])
        self.activationF=activationF
    
        if nNodesNext==0:
            self.weights=None
        else:
            self.weights=np.random.normal(0, 1, size=(nNodesCurrent,nNodesNext))

In [173]:
class NeuralNet:
    def __init__(self, totalLayers, noNodesList, activationFunctions):
        self.totalLayers=totalLayers
        self.noNodesList=noNodesList
        self.layers = []
        for i in range(totalLayers):
            currentLayerNodes=noNodesList[i]
            if i!=totalLayers-1:
                nextLayerNodes=noNodesList[i+1]
                ith_Layer=Layer(currentLayerNodes,nextLayerNodes,activationFunctions[i])
            else:
                ith_Layer=Layer(currentLayerNodes,0,activationFunctions[i])
            self.layers.append(ith_Layer)#append output layer as none

    def trainNetwork(self, data,outputLabels, batchSize, epochs, learningRate):
        self.learningRate=learningRate
        self.batchSize=batchSize;
        
        #normalize data
        data=((data-data.min(axis=0))/(data.max(axis=0)-data.min(axis=0)))
                
        for x in range(epochs):
            i=0  
            while i<len(data):
                self.error=0
                self.forwardPropo(data[i:i+batchSize])#input
                self.calculateError(outputLabels[i:i+batchSize])#output
#                 print("==============Batch",i,"================")
                self.back_pass(outputLabels[i:i+batchSize])
                i+=batchSize
            self.error /= batchSize
            print("Error: ", self.error)
          
    def forwardPropo(self, inputs):
        self.layers[0].activations =inputs
        for i in range(self.totalLayers-1):
#             print(self.layers[i].activations.shape,self.layers[i].weights.shape,)
            temp=np.matmul(self.layers[i].activations,self.layers[i].weights)  
#           print("==============================")
            if self.layers[i+1].activationF == "sigmoid":
                self.layers[i+1].activations = self.sigmoid(temp)
            elif self.layers[i+1].activationF == "softmax":
                self.layers[i+1].activations = self.softmax(temp)
            elif self.layers[i+1].activationF == "relu":
                self.layers[i+1].activations = self.relu(temp)
            elif self.layers[i+1].activationF == "tanh":
                self.layers[i+1].activations = self.tanh(temp)
            else:
                self.layers[i+1].activations = temp
            # print(self.layers[self.totalLayers-1].activations)
        
    def calculateError(self,labels):
#         print(labels.shape)
        if len(labels[0]) != self.layers[self.totalLayers-1].nodesNo:
            print ("Error: Label is not of the same shape as output layer.")
            print("Label: ", len(labels), " : ", len(labels[0]))
            print("Out: ", len(self.layers[self.totalLayers-1].activations), " : ", len(self.layers[self.totalLayers-1].activations[0]))
            return
        self.error += np.negative(np.sum(np.multiply(labels, np.log(self.layers[self.totalLayers-1].activations))))
    
    def back_pass(self, labels):
        # if self.cost_function == "cross_entropy" and self.layers[self.num_layers-1].activation_function == "softmax":
        targets = labels
        i = self.totalLayers-1
        y = self.layers[i].activations
        
        delta=(y-targets)#/targets.shape[0]
        deltaw = np.dot(self.layers[i-1].activations.T, delta)/self.batchSize
        new_weights = self.layers[i-1].weights - self.learningRate * deltaw
        for i in range(i-1, 0, -1):
            
            sigPrime = self.sigmoidDerivative(self.layers[i].activations)
            delta=np.multiply(sigPrime,delta.dot(self.layers[i].weights.T))
            deltaw = np.dot(self.layers[i-1].activations.T, delta)/self.batchSize

            self.layers[i].weights = new_weights
            new_weights = self.layers[i-1].weights - self.learningRate * deltaw
        self.layers[0].weights = new_weights
            
    def check_accuracy(self, inputs, labels):
        self.batchSize = len(inputs)
        self.forwardPropo(inputs)
        a = self.layers[self.totalLayers-1].activations
        print(len(a))
        total=0
        correct=0
        for i in range(len(a)):
            total += 1
            al = a[i].tolist()
            if labels[i][al.index(max(al))] == 1:
                correct += 1
        print(correct)
        print("Accuracy: ", correct*100/total)
    
    def sigmoid(self, x):
        return np.divide(1, np.add(1, np.exp(np.negative(x))))
    
    def sigmoidDerivative(self,x):
        return (self.sigmoid(x)*(1-self.sigmoid(x)))
    
    def relu(self, x):
        x[x<0] = 0
        return x
    
    def relu_derivative(X):
        return 1. * (X > 0)
    
    def softmax(self, x):
        exp = np.exp(x)
        if isinstance(x[0], np.ndarray):
            return exp/np.sum(exp, axis=1, keepdims=True)
        else:
            return exp/np.sum(exp, keepdims=True)

    def tanh(self, x):
        return np.tanh(x)
    
    def tanhDerivative(x):
        return 1.0 - np.tanh(x) ** 2

In [174]:
data=pd.read_csv("../input/apparel-trainval.csv").values

In [175]:
def getOneHotLabels(data,k):
    one_hot_labels = np.zeros((len(data), k))
    for i in range(len(data)):  
        one_hot_labels[i,data[i,0]] = 1
    return one_hot_labels

In [178]:
train,test=splitTrainTest(data,80)
oneHotLabelsTrain=getOneHotLabels(train,10)
oneHotLabelsTest=getOneHotLabels(test,10)
print(train.shape)
trainInputs=train[:,1:]
testInputs=test[:,1:]
print(trainInputs.shape)
# nn=NeuralNet(4,[784,16,16,10],["sigmoid","sigmoid","sigmoid","softmax"])
nn=NeuralNet(3,[784,500,10],[None,"sigmoid","softmax"])

(48000, 785)
(48000, 784)


In [179]:
nn.trainNetwork(trainInputs,oneHotLabelsTrain,100,200,0.01)
nn.check_accuracy( testInputs, oneHotLabelsTest)

Error:  2.4485487329703775
Error:  1.7617810079175198
Error:  1.4497582425450468
Error:  1.2876362370135856
Error:  1.161496219885037
Error:  1.0782030744162285
Error:  1.0235548451049408
Error:  0.9940743116865158
Error:  0.9912080446361867
Error:  1.0000475346689386
Error:  1.0170530030821012
Error:  1.0404956780255712
Error:  1.0642051107613772
Error:  1.0736596086716426
Error:  1.0632503686339518
Error:  1.0565771489896063
Error:  1.071497720014611
Error:  1.0997860101916666
Error:  1.1227154983194367
Error:  1.1199742671336042
Error:  1.085593205661077
Error:  1.0325935487782774
Error:  0.9818501406645294
Error:  0.9458598183224634
Error:  0.9278020493195618
Error:  0.9268612663118698
Error:  0.9423379842004814
Error:  0.9664177221103071
Error:  0.9941387641148526
Error:  1.0226459090703204
Error:  1.043614457702835
Error:  1.052339071199499
Error:  1.051673838648337
Error:  1.0471656917433347
Error:  1.0439901500293585
Error:  1.0353168747053474
Error:  1.0168772292786097
Error: 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:96: RuntimeWarning: overflow encountered in exp
